In [51]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [52]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001

# image preprocessing modules
# data augmentation
transform = transforms.Compose([transforms.Pad(4),
                                transforms.RandomHorizontalFlip(), # default p=0.5
                                transforms.RandomCrop(32),
                                transforms.ToTensor()
                                ])

In [47]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True,
                                             transform=transform,
                                             download=True
                                             )

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False,
                                            transform=transforms.ToTensor()
                                            ) 
             
# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2
                                           )

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2
                                          )

# 3 x 3 Convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)
    
# Residual block
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out1 = self.conv1(x)
        out1 = self.bn1(out1)
        out1 = self.relu(out1)
        out2 = self.conv2(out1)
        out2 = self.bn2(out2)
        if self.downsample:
            residual = self.downsample(x)
        out = out2 + residual
        out = self.relu(out)
        return out

# Residual NN
class ResNN(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNN, self).__init__()
        self.in_channels = 16           # num_kernels
        self.conv = conv3x3(3, 16)      # RGB --> num_kernels
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2) # layers: num of repeated blocks
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    # construct more layers composed by blocks
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels)
                                       )
        
        layers = []

        # here, block is from 'ResBlock' instance
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):  # blocks: num of repeated resblock
            layers.append(block(out_channels, out_channels))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

Files already downloaded and verified


In [53]:
model = ResNN(ResBlock, [2, 2, 2, 2]).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# update learning rate
def update_lr(optimizer, lr):
    for para in optimizer.param_groups:
        para['lr'] = lr 



注：使用衰减learning rate，刚开始epoch用大一点lr快速下降，后期的epoch中用小lr慢慢搜索

In [54]:
# train model
total_step = len(train_loader)
print('Each epoch has {} iterations'.format(total_step))
cur_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward
        output = model(images)
        loss = criterion(output, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        # decay learning rate
        if (epoch+1) % 20 == 0:
            cur_lr /= 3
            update_lr(optimizer, cur_lr)

500


KeyboardInterrupt: ignored

In [ ]:
# test model
model.eval()            
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predict = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predict == labels).sum().item()

    print('Accuracy of model is {}'.format(100*correct/total))

# save model
# torch.save(model.state_dict(), 'resnet.pkl')